In [1]:
import os,sys
import numpy as np
from collections import namedtuple
import tqdm
import glob
import math
import random
import inspect
import os.path as osp
from pathlib import Path
import itertools
from itertools import chain
import numpy as np
import pandas as pd
import multiprocessing
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sys, os
from importlib import reload

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.data import Data, DataLoader, DataListLoader
from torch_geometric.nn import EdgeConv, global_mean_pool, DataParallel
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data,Dataset
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv
import h5py

In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../')))

import models_torch.models as models
reload(models)
import models_torch.losses as losses
reload(losses)
import utils_torch.preprocessing as prepr
reload(prepr)
import utils_torch.plot_util as plot
reload(plot)
import utils_torch.train_util as train
reload(train)
import graph_data.graph_data as graph_data
reload(graph_data)

<module 'graph_data.graph_data' from '/eos/home-n/nchernya/MLHEP/AnomalyDetection/ADgvae/graph_data/graph_data.py'>

In [3]:
data_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
dataset = graph_data.GraphDataset(root=data_dir,n_jets=1e4)
    
use_generator = False
if use_generator:
    validation_split = 0.2
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    if dataset_size > 2:
        split = int(np.floor(validation_split * dataset_size))
    else: 
        split = 1
    print(dataset_size,split)
    random_seed= 1001

    train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
                                                             generator=torch.Generator().manual_seed(random_seed))
    print("train subset dim:", len(train_subset))
    print("validation subset dim", len(val_subset))
    dataloaders = {
        'train':  DataLoader(train_subset, batch_size=128, shuffle=True),
        'val':   DataLoader(val_subset, batch_size=128, shuffle=True)
    }
    print("train_dataloader dim:", len(dataloaders['train']))
    print("val dataloader dim:", len(dataloaders['val']))
else : 
    in_memory_datas = dataset.return_inmemory_data() 

In [4]:
pf_cands,jet_prop = dataset.pf_cands,dataset.jet_prop
pf_cands_t = torch.cat([torch.tensor(pf_cands[i], dtype=torch.float) for i in range(len(pf_cands))])

In [33]:
sys.path.append(os.path.abspath(os.path.join('../../')))

import vande.util.util_plotting as vande_plot
reload(vande_plot)

<module 'vande.util.util_plotting' from '/eos/home-n/nchernya/MLHEP/AnomalyDetection/vande/util/util_plotting.py'>

In [45]:
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
jet_feats = 'N_constituents,M,Pt,Eta,Phi'.split(',')


save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
vande_plot.plot_features(pf_cands_t.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)
vande_plot.plot_features(jet_prop[:,0:-1], jet_feats ,'Normalized' , 'QCD', plotname='{}plot_jet_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [45]:
#scaler = standardize(train_subset) # I dont think that this works for the dataset implementation as it is done now, implement scaler as part of dataset ? 
scaler = prepr.standardize(in_memory_datas) 

dataloaders = {
    'train':  DataLoader(in_memory_datas, batch_size=128, shuffle=True)
    }
print("train_dataloader dim:", len(dataloaders['train']))

#dataset.get(0).u

train_dataloader dim: 8


In [46]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
multi_gpu = False #torch.cuda.device_count()>1

In [49]:
#loss
#loss_ftn_obj = LossFunction('mse_coordinates', device=device)
loss_ftn_obj = losses.LossFunction('mse', device=device)

# model
input_dim = 7
output_dim = 7#4
big_dim = 32
hidden_dim = 2
model = models.EdgeNet(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = 10e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, threshold=1e-6)

model.to(device)


EdgeNet(
  (batchnorm): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
    (5): ReLU()
  ))
  (decoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=7, bias=True)
  ))
)

In [58]:
# Training loop
n_epochs = 80
stale_epochs = 0
loss = 999999
train_losses = []
for epoch in range(0, n_epochs):
    #loss = train(model, optimizer, loader, len(datas), 128, loss_ftn_obj)
    loss = train.train(model, optimizer, dataloaders['train'], len(dataloaders['train'].dataset), dataloaders['train'].batch_size, loss_ftn_obj)
    train_losses.append(loss)
    print('Epoch: {:02d}, Training Loss:   {:.4f}'.format(epoch, loss))


  0%|          | 0/7.8125 [00:00<?, ?it/s]
train loss = 1.1026058:   0%|          | 0/7.8125 [00:04<?, ?it/s]
train loss = 1.1026058:   0%|          | 0/7.8125 [00:04<?, ?it/s]
train loss = 1.1026058:  13%|█▎        | 1/7.8125 [00:04<00:30,  4.54s/it]
train loss = 0.9385760:  13%|█▎        | 1/7.8125 [00:04<00:30,  4.54s/it]
train loss = 0.9385760:  13%|█▎        | 1/7.8125 [00:04<00:30,  4.54s/it]
train loss = 0.9385760:  26%|██▌       | 2/7.8125 [00:04<00:18,  3.22s/it]
train loss = 1.0185511:  26%|██▌       | 2/7.8125 [00:04<00:18,  3.22s/it]
train loss = 1.0185511:  26%|██▌       | 2/7.8125 [00:04<00:18,  3.22s/it]
train loss = 1.0185511:  38%|███▊      | 3/7.8125 [00:04<00:11,  2.29s/it]
train loss = 1.0707859:  38%|███▊      | 3/7.8125 [00:04<00:11,  2.29s/it]
train loss = 1.0707859:  38%|███▊      | 3/7.8125 [00:04<00:11,  2.29s/it]
train loss = 1.0707859:  51%|█████     | 4/7.8125 [00:04<00:06,  1.63s/it]
train loss = 0.8313523:  51%|█████     | 4/7.8125 [00:05<00:06,  1.63s/i

Epoch: 00, Training Loss:   0.9102



train loss = 0.7560380:  13%|█▎        | 1/7.8125 [00:00<00:00,  8.14it/s]
train loss = 0.7560380:  13%|█▎        | 1/7.8125 [00:00<00:00,  8.14it/s]
train loss = 0.7560380:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.38it/s]
train loss = 0.7405478:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.38it/s]
train loss = 0.7405478:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.38it/s]
train loss = 0.7405478:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.53it/s]
train loss = 0.7379527:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.53it/s]
train loss = 0.7379527:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.53it/s]
train loss = 0.7379527:  51%|█████     | 4/7.8125 [00:00<00:00,  8.63it/s]
train loss = 0.8141848:  51%|█████     | 4/7.8125 [00:00<00:00,  8.63it/s]
train loss = 0.8141848:  51%|█████     | 4/7.8125 [00:00<00:00,  8.63it/s]
train loss = 0.8141848:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.60it/s]
train loss = 0.6988147:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.60it/s]
train loss = 0.6988147: 

Epoch: 01, Training Loss:   0.7224



train loss = 0.6363030:  13%|█▎        | 1/7.8125 [00:00<00:00,  8.14it/s]
train loss = 0.6363030:  13%|█▎        | 1/7.8125 [00:00<00:00,  8.14it/s]
train loss = 0.6363030:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.48it/s]
train loss = 0.6769050:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.48it/s]
train loss = 0.6769050:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.48it/s]
train loss = 0.6769050:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.77it/s]
train loss = 0.5703521:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.77it/s]
train loss = 0.5703521:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.77it/s]
train loss = 0.5703521:  51%|█████     | 4/7.8125 [00:00<00:00,  9.00it/s]
train loss = 0.5411949:  51%|█████     | 4/7.8125 [00:00<00:00,  9.00it/s]
train loss = 0.5411949:  51%|█████     | 4/7.8125 [00:00<00:00,  9.00it/s]
train loss = 0.5411949:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.64it/s]
train loss = 0.6736951:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.64it/s]
train loss = 0.6736951: 

...

Epoch: 77, Training Loss:   0.0427



train loss = 0.0267673:  13%|█▎        | 1/7.8125 [00:00<00:00,  8.09it/s]
train loss = 0.0267673:  13%|█▎        | 1/7.8125 [00:00<00:00,  8.09it/s]
train loss = 0.0267673:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.21it/s]
train loss = 0.0632067:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.21it/s]
train loss = 0.0632067:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.21it/s]
train loss = 0.0632067:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.41it/s]
train loss = 0.0342462:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.41it/s]
train loss = 0.0342462:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.41it/s]
train loss = 0.0342462:  51%|█████     | 4/7.8125 [00:00<00:00,  8.60it/s]
train loss = 0.0334786:  51%|█████     | 4/7.8125 [00:00<00:00,  8.60it/s]
train loss = 0.0334786:  51%|█████     | 4/7.8125 [00:00<00:00,  8.60it/s]
train loss = 0.0334786:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.72it/s]
train loss = 0.0469487:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.72it/s]
train loss = 0.0469487: 

Epoch: 78, Training Loss:   0.0400



train loss = 0.0262361:  13%|█▎        | 1/7.8125 [00:00<00:00,  7.73it/s]
train loss = 0.0262361:  13%|█▎        | 1/7.8125 [00:00<00:00,  7.73it/s]
train loss = 0.0262361:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.10it/s]
train loss = 0.0501203:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.10it/s]
train loss = 0.0501203:  26%|██▌       | 2/7.8125 [00:00<00:00,  8.10it/s]
train loss = 0.0501203:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.44it/s]
train loss = 0.0377088:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.44it/s]
train loss = 0.0377088:  38%|███▊      | 3/7.8125 [00:00<00:00,  8.44it/s]
train loss = 0.0377088:  51%|█████     | 4/7.8125 [00:00<00:00,  8.73it/s]
train loss = 0.0371194:  51%|█████     | 4/7.8125 [00:00<00:00,  8.73it/s]
train loss = 0.0371194:  51%|█████     | 4/7.8125 [00:00<00:00,  8.73it/s]
train loss = 0.0371194:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.97it/s]
train loss = 0.0375429:  64%|██████▍   | 5/7.8125 [00:00<00:00,  8.97it/s]
train loss = 0.0375429: 

Epoch: 79, Training Loss:   0.0400


8000

In [79]:
inverse_standardization = False
plot.plot_reco_for_loader(model, dataloaders['train'], device, scaler, inverse_standardization, 'test_train', osp.join(save_dir, 'reconstruction_post_train', 'train_reco_all_std'), 'all')
